In [1]:
import os
import sys
import time
import yaml
import numpy as np
import torch
import mmengine
from torch.utils.data import DataLoader
os.chdir('../')
from new_model.model import TrajectoryModel
from new_tools.dataset import TrajectoryDataset

In [2]:
dataset_path = './data/CODA/'
dataset_name = 'coda'
config = yaml.load(open('./configs/coda.yaml', 'r'), Loader=yaml.FullLoader)
train_dataset = TrajectoryDataset(dataset_path, dataset_name, 'train', class_balance=True)
val_dataset = TrajectoryDataset(dataset_path, dataset_name, 'val')
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, collate_fn=val_dataset.collate_fn)
model = TrajectoryModel(num_class=config['num_class'], in_size=2, obs_len=config['obs_len'], pred_len=config['pred_len'], 
                        embed_size=config['embed_size'], num_decode_layers=config['num_decode_layers'], scale=0.1)
model.cuda()

TrajectoryModel(
  (encoder): Encoder(
    (self_embedding): ModuleList(
      (0-2): 3 x Linear(in_features=80, out_features=128, bias=True)
    )
    (nei_embedding): ModuleList(
      (0-3): 4 x Linear(in_features=32, out_features=128, bias=True)
    )
  )
  (decoder): Decoder(
    (decoderLayers): ModuleList(
      (0-1): 2 x DecoderLayer(
        (attns): ModuleList(
          (0-1): 2 x TransformerBlock(
            (attn): MultihHeadAttention(
              (w_key): Linear(in_features=128, out_features=128, bias=True)
              (w_query): Linear(in_features=128, out_features=128, bias=True)
              (w_value): Linear(in_features=128, out_features=128, bias=True)
              (fc_out): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (norm1): LayerNorm()
            (norm2): LayerNorm()
            (feed_forward): FeedForwardLayer(
              (w1): Linear(in_features=128, out_feature

In [3]:
model.eval()
with torch.no_grad():
    for data_batch in val_dataloader:
        data_batch = [tensor.cuda() for tensor in data_batch]
        obs, futures, neis, nei_masks, self_labels, nei_labels, refs, rot_mats = data_batch
        preds, scores, init_traj = model(obs, neis, nei_masks, self_labels, nei_labels)
        break

In [5]:
preds.shape, scores.shape, init_traj.shape

(torch.Size([128, 49, 24, 2]),
 torch.Size([128, 49]),
 torch.Size([3, 49, 24, 2]))

In [7]:
model.train()
for data_batch in train_dataloader:
    data_batch = [tensor.cuda() for tensor in data_batch]
    obs, futures, neis, nei_masks, self_labels, nei_labels, refs, rot_mats = data_batch
    preds, scores, init_traj = model(obs, neis, nei_masks, self_labels, nei_labels)
    break

In [9]:
obs, futures, neis, nei_masks, self_labels, nei_labels, refs, rot_mats = data_batch

In [ ]:
obs.shape, futures.shape, neis.shape, nei_masks.shape, self_labels.shape, nei_labels.shape, refs.shape, rot_mats.shape

In [ ]:
preds.size(-2)

In [3]:
train_dataset = TrajectoryDataset(dataset_path, dataset_name, 'train', class_balance=True)

In [ ]:
len(train_dataset)

In [ ]:
num = [0, 0, 0]
for data in train_dataset.data_list:
    num[data['label']] += 1
num